# Draft Set Design

## Design Goals

- Players will draft fully playable legal decks - respecting influence
    - IDs may need to be modified to make this possible
- Coallation of packs can help smooth the draft experience, but should not be super predictable
- “Rarity” should be included, and indicated to players in some fashion
- Cards should have some cross-archetype competition, parasitic cards should largely be avoided, or allow some cross canabalization
- Probably slightly lower power than current standard, perhaps startup level, but looking to make gameplay experiences fairly diverse

# Corp Design

I’d like to just provide people with PriRec’s - which means players will need to draft 18-20 points of agendas (maybe 22 if MoH is included). I’d like to keep the draft moving, but hopefully signposting and factions help push that. Supporting assets, glacier, FA, combo, and Tag-N-Bag is a big ask for a pool.

Corp decks typically will have 8-12 agendas. If players are given 60 cards that means 1/5 cards will be agendas. Econ cards typically compose 9-15 cards (though lots of overlap) so probably 1/5 cards need to be econ, perhaps slightly more. Ice bottoms out at 9 and can double to 18, so probably 1/5 of cards should be ice, 1/5 of cards should gameplan cards, and 1/5 should be a mix of value assets and additional ice.
8 players * 60 cards = 480 cards - might want a cardpool for ~10 players to make it more distinct from pod to pod - so 600 cards. Will want lots of duplicates. But going by those ratios:

120 Agendas (average points 2/agenda)

120 Econ cards (3+ credit value)

120 Ice

120 Build around pieces

120 flex cards (Ice/Assets/Upgrades)

Also need to balance influence and agendas -
each faction needs enough agendas available to support two players, so 40 points across 30 agendas - Neutrals reduce this need a bit

In [2]:
import pandas as pd
import requests

card_data = requests.request(url="https://netrunnerdb.com/api/2.0/public/cards",method="GET").json()['data']
cards_df = pd.DataFrame(card_data)
cards_df = cards_df.query("pack_code not in ['draft','tdc']")

trashorbusto_data = requests.request(url="https://trash-or-busto.herokuapp.com/api/rankings",method="GET").json()
trashorbusto_corp_df = pd.DataFrame(trashorbusto_data['corp_cards'])
trashorbusto_corp_df['rating'] = pd.to_numeric(trashorbusto_corp_df['rating']).round(0)


In [2]:
pd.merge(
    cards_df[cards_df['type_code'] == 'agenda'],
    trashorbusto_corp_df[trashorbusto_corp_df['rating'] > 1200],
    how='inner',
    left_on='code',
    right_on='nrdb_key'
).to_csv("CorpAgendasGreater1200.csv")


## Thoughts After Looking at Agendas

I don't want to include PE in the pool - I think it's a bit parasitic. That doesn't mean net damage won't be part of it, but I think taking it out of the pool can hopefully give net damage a slightly different look

Have 84 points of Neutral Corp Agendas on the first pass -> so would need about 30 points/faction
Generally finding ~70 points of agendas fit easily, with ~10-20 points that support specific gameplans
So will tighten things up. Need to make a decision about 3x Astro

Archtypes defined by agendas:
- Brain Rewiring Kill (would require CI/PD/Cybernetics Court)
- EffComm Combo (SfSS, DediRecoc)
- Regulatory Capture (Is this parasitic bad pub Outfit? - can you work this without the ID?)
- Obokata/Blood in the Water (Handsize matters)

# Archtype Overviews:

- NBN
    - Fast Advance (NEH/Astroscript/SSCG/SfSS/VladGrid)
    - Asset based
    - Tag-N-Bag
    - Glacier
- HB
    - ~~Brain Rewiring Combo~~
    - EffComm Combos
    - Never-advance (Jeeves/Seamless Launch)
    - Core Damage
    - Fast Advance (Biotic/SSCG)
    - Sportsmetal - agenda feeders
- Jinteki
    - AgInf/RP glacier
    - Tempo/Grinder traps
    - IG Combo (probably want to lean on the combo)
    - ~~PE/PU Grinder~~
- Weyland
    - Gagarin Assets
    - Argus/Tag-N-Bag
    - Outfit FA
    - Ob Combo Lord
    - ~~Blue Sun~~

In [3]:
pd.set_option("display.max_rows",None)
labeled_agendas = pd.read_csv("CorpAgendasGreater1200_handlabeled.csv")[['faction_code','title','rating','nrdb_key', 'Prospective Copies']].dropna().query("`Prospective Copies` > 0")

In [4]:
pd.set_option("display.max_rows",10)


In [3]:
ice = pd.merge(
    cards_df[cards_df['type_code'] == 'ice'],
    trashorbusto_corp_df,
    how='inner',
    left_on='code',
    right_on='nrdb_key'
)


# What does the pack composition look like:
## 12 card pack
- 2.5 agendas
- 3 ice
- 2 operations
- 2 assets
- 2 flex cards

Distribution of rarities goes something like:
- 


After some some more reflection - using rarity rather than copies, and then combining that with pack-seeding (fixed slots for agendas/ice/econ/etc) makes more sense that doing fixed copies. So what should the rarities be, and what is the purpose of those rarities.

- Common: Low influence cards that are playable in most decks in faction
- Uncommon: Low influence cards that can be good particular archtypes (signal lanes, so need to support the baseline engine of the archtype)
- Rare: High influence cards that are good in faction
- Mythic: High influence cards that are good in particular archtypes or Haymakers (big potential build arounds)

In [4]:
# Heuristic binning of Ice

# Algorigthm will account for two things - the TorB rating, and the influence cost of the card.
# First filter out cards that are in the bottom 20% of TorB ratings.
# Put low influence cards (0-2) into the "low" bin.
# Put high influence cards (3-5) into the "high" bin.
# Commons should be cards in the ~50% range of TorB ratings that are in the low influence bin.
# Uncommons should be cards in the 75th percentile range of TorB ratings that are in the low influence bin.
# Faction Commons should be cards in the ~50% range of TorB ratings that are in the high influence bin.
# Faction Uncommons should be cards in the 60th percentile range of TorB ratings that are in the high influence bin.
# Faction Rares should cards in the 80% percentile range of TorB ratings that are in the high influence bin.

# This is a heuristic, and will need to be adjusted as we get more data.

ice[['stripped_title','faction_code','rating','faction_cost','keywords']].quantile([0.2,0.5,0.75,0.8],numeric_only=True)
ice['stripped_title']= ice['stripped_title'].str.replace(";","") # handlinge TL;DR so it doesn't split the csv
# 0.2    0.5    0.75   0.8
# 1294   1452   1562   1642
ice_abbreviated = ice[['stripped_title','faction_code','rating','faction_cost','keywords', 'stripped_text']].copy()

special_subtypes = ['Bioroid','Tracer','Harmonic']

def make_rarity_ice(row):
    if row['rating'] < 1350:
        return "Trash"
    
    elif row['faction_cost'] < 3:
        for subtype in special_subtypes:
            if subtype in row['keywords']:
                return "Faction Common"
        if row['rating'] < 1600:
            return "Common"
        else:
            return "Rare"
    else:
        if row['rating'] < 1600:
            return "Faction Common"
        else:
            return "Faction Rare" 

ice_abbreviated['rarity']= ice_abbreviated.apply(lambda x:make_rarity_ice(x),axis=1)
ice_abbreviated.to_csv("corp_spreadsheets/ice_raw_rarities.csv", index=False)
pd.set_option("display.max_rows",None)

ice_abbreviated['type'] = ice_abbreviated['keywords'].str.split('-').str[0].str.strip()
ice_abbreviated.groupby(['type','rarity'])['stripped_title'].count().reset_index()
ice_abbreviated['etr']= ice_abbreviated["stripped_text"].str.contains("[E,e]nd the run")
ice_abbreviated[(ice_abbreviated['etr']) & (ice_abbreviated['type'] == 'Sentry')]
# Looks like faction Uncommon and Uncommon are super sparse. Also synergy cards (Harmonics/Bioroids) are slightly misclasified
# Some more advanced heuristics - Bioriod and Harmonic should be faction commons (except Eli 1.0)
# NBN traces/observer should be faction commons
# Jinteki AP should be faction commons

# Revised rarities - Common (cross faction cards), Faction Commons, Rares - both faction and neutral high power cards
# These probably are your common/uncommon/rare bins - potentially using TorB rating for Mythics - though probably hand coded

# for 6/21 do this for all corp cards, and plan on trying out pack seeding algorithms

,stripped_title,faction_code,rating,faction_cost,keywords,stripped_text,rarity,type,etr
7,Draco,neutral-corp,1274.0,0,Sentry - Tracer,"When you rez Draco, you may pay X credits to p...",Trash,Sentry,True
15,Uroboros,nbn,1291.0,2,Sentry - Tracer,"Subroutine Trace[4]. If successful, the Runner...",Trash,Sentry,True
31,Muckraker,nbn,1270.0,3,Sentry - Tracer - Illicit,"When you rez Muckraker, take 1 bad publicity. ...",Trash,Sentry,True
35,Fenris,haas-bioroid,1396.0,2,Sentry - AP - Illicit,"When you rez this ice, take 1 bad publicity. S...",Common,Sentry,True
39,Shinobi,jinteki,1356.0,3,Sentry - Tracer - AP - Illicit,"When you rez Shinobi, take 1 bad publicity. Su...",Faction Common,Sentry,True
47,Guard,neutral-corp,1479.0,0,Sentry,Guard cannot be bypassed. Subroutine End the run.,Common,Sentry,True
48,Rainbow,neutral-corp,1357.0,0,Sentry - Code Gate - Barrier,Subroutine End the run.,Common,Sentry,True
68,Troll,nbn,1362.0,2,Sentry,"When the Runner encounters Troll, Trace[2]. If...",Common,Sentry,True
73,Orion,weyland-consortium,1484.0,3,Sentry - Code Gate - Barrier,Orion can be advanced and its rez cost is lowe...,Faction Common,Sentry,True
93,Tour Guide,weyland-consortium,1799.0,2,Sentry,"Tour Guide gains ""Subroutine End the run."" for...",Rare,Sentry,True


In [14]:
agendas = pd.merge(
    cards_df[cards_df['type_code'] == 'agenda'],
    trashorbusto_corp_df,
    how='inner',
    left_on='code',
    right_on='nrdb_key'
)
agendas[['stripped_title','faction_code','rating','faction_cost','keywords','agenda_points']].quantile([0.2,0.5,0.75,0.8],numeric_only=True)
normal_ratios = {2:1, 3:2, 4:2, 5:3, 6:-1}
def make_rarity_agenda(row):
    if row['rating'] < 1350:
        return "Trash"
    if row['faction_code'] == 'neutral-corp':
        if row['rating'] < 1600:
            if row['agenda_points'] == normal_ratios[row['advancement_cost']]:
                return "Common"
            else:
                return "Trash"
        else:
            return "Rare"
    else:
        if row['rating'] < 1600:
            if row['agenda_points'] == normal_ratios[row['advancement_cost']]:
                return "Faction Common"
            else:
                return "Faction Trash"
        else:
            return "Faction Rare"
        
agendas['rarity']= agendas.apply(lambda x:make_rarity_agenda(x),axis=1)
agendas[['faction_code','rarity','stripped_title',"agenda_points","advancement_cost", 'rating']].to_csv("corp_spreadsheets/agendas_raw_rarities.csv", index=False)

In [8]:
assets = pd.merge(
    cards_df[cards_df['type_code'] == 'asset'],
    trashorbusto_corp_df,
    how='inner',
    left_on='code',
    right_on='nrdb_key'
)
assets[['stripped_title','faction_code','rating','faction_cost','keywords']].quantile([0.2,0.5,0.6,0.8],numeric_only=True)

def make_rarity_asset(row):
    if row['rating'] < 1300:
        return "Trash"
    if row['faction_code'] == 'neutral-corp':
        if row['rating'] < 1500:
            return "Common"
        else:
            return "Rare"
    else:
        if row['rating'] < 1500:
            return "Faction Common"
        else:
            return "Faction Rare"
        
assets['rarity']= assets.apply(lambda x:make_rarity_asset(x),axis=1)
assets[['faction_code','rarity','stripped_title', 'rating']].to_csv("corp_spreadsheets/assets_raw_rarities.csv", index=False)

In [11]:
upgrades = pd.merge(
    cards_df[cards_df['type_code'] == 'upgrade'],
    trashorbusto_corp_df,
    how='inner',
    left_on='code',
    right_on='nrdb_key'
)
upgrades[['stripped_title','faction_code','rating','faction_cost','keywords']].quantile([0.2,0.5,0.6,0.8],numeric_only=True)
def make_rarity_upgrade(row):
    if row['rating'] < 1300:
        return "Trash"
    if row['faction_code'] == 'neutral-corp':
        if row['rating'] < 1500:
            return "Common"
        else:
            return "Rare"
    else:
        if row['rating'] < 1500:
            return "Faction Common"
        else:
            return "Faction Rare"

upgrades['rarity']= upgrades.apply(lambda x:make_rarity_upgrade(x),axis=1)
upgrades[['faction_code','rarity','stripped_title', 'rating']].to_csv("upgrades_raw_rarities.csv", index=False)


In [13]:
operations = pd.merge(
    cards_df[cards_df['type_code'] == 'operation'],
    trashorbusto_corp_df,
    how='inner',
    left_on='code',
    right_on='nrdb_key'
)
operations[['stripped_title','faction_code','rating','faction_cost','keywords']].quantile([0.2,0.5,0.6,0.8],numeric_only=True)
def make_rarity_operation(row):
    if row['rating'] < 1300:
        return "Trash"
    if row['faction_code'] == 'neutral-corp':
        if row['rating'] < 1500:
            return "Common"
        else:
            return "Rare"
    else:
        if row['rating'] < 1500:
            return "Faction Common"
        else:
            return "Faction Rare"
operations['rarity']= operations.apply(lambda x:make_rarity_operation(x),axis=1)
operations[['faction_code','rarity','stripped_title', 'rating']].to_csv("corp_spreadsheets/operations_raw_rarities.csv", index=False)

# Calculating Unique cards in pool
Can use the above code to calculate based on a number of unique cards in the pool, and the number of total draws what the probability of seeing a given number of copies is.

For reference MtG allows about a 5% chance you'd see more than 4 copies of a common card across the whole pool.
And about a 7% chance you'd see 3 copies of an uncommon
And about a 7% chance you'd see 2 copies of a rare



In [1]:
unique_commons_per_color = 10
colors = 5
draws_per_pack = 1
packs_per_draft = 3
cards_per_color = packs_per_draft * draws_per_pack * 8 / colors
cards_per_color

4.8

# Rarity Meanings
I think commons I want to have between 4-8 of a given common in the pool. Commons should include fundementals for making the game work and cards that go in a variety of decks. Examples of high power commons include GFI and Hedge Fund/Sure Gamble.
For uncommons there shoul be between 1-3 of a given uncommon. Uncommons should by synergistic build-around tools. Examples can be things like SfSS, or cards that are good in a particular archetype like Jeeves or Seamless Launch.
For rares there should be between 0-2 of a given rare. Rares should be high impact cards, that fit in specific decks.

## 13 card Pack
Going to a 13 card Corp pack because it allows each player to draft 65 cards, allowing about 15 chaff cards.

The breakdown of the pack rarities
2 Rares
4 Uncommons
7 Commons (with a 5% chance of a common upgraded to a rare?)

Card Type gurantees:
- 2 Agendas
- 3 Ice
- 2 Operations
- 2 Assets
- 1 Upgrade
- 3 Flex Cards

So now I need to figure out the cardpool size for each rarity. Given the above targets, we can calculate the number of cards in each rarity. To calculate it I want a 95% chance that a pool of cards will have the number of cards I want. So I can calculate the probability using the binomal distribution, because I'm assuming cards are replaced each time they're pulled. So to calculate the probability of a specific number of cards being pulled I can use the following formula:
(nCk)(p^k)(1-p)^(n-k)
where n is the number of draws, k is the number of successes, and p is the probability of success.
p will be 1/number of cards in the pool. Given than n is 7 * 5 * 8 for commons, and 4 * 5 * 8 for uncommons, and 2 * 5 * 8 for rares, I can calculate the number of cards I need to have a 95% chance of getting the number of cards I want.

Approach will be working up from a 1 card pool until I get the numbers desired from above, probably favoring slightly larger cardpools for variety.

In [459]:
from math import factorial, comb
def binomial(n, k):
    return factorial(n + k - 1) // (factorial(k) * factorial(n - 1))

def calculate_copies_probability(unique_cards: int, draws: int, copies: int = 3):
    """
    Calculates the probability of drawing at least one copy of a card in a deck of `draws` cards, where there are `unique_cards` unique cards in the deck,
    and each card has `max_copies` copies in the deck.

    Copies get replaced after each draw.
    """
    p = 1/unique_cards
    return (comb(draws, copies) * (p ** copies)) * ((1 - p) ** (draws- copies))

def sum_probabilities(unique_cards: int, draws: int, copies: int = 3):
    return sum([calculate_copies_probability(unique_cards, draws, copies) for copies in range(0, copies + 1)])

def compute_probability(unique_cards: int, draws: int, copies: int = 3):
    return [calculate_copies_probability(unique_cards, draws, copies) for copies in range(0, copies + 1)]

def min_copies_probability(unique_cards: int, draws:int, min_copies: int = 1,  max_copies: int = 10):
    return sum([calculate_copies_probability(unique_cards, draws, copies) for copies in range(min_copies, max_copies + 1)])

total_commons = 8 * 5 * 8 # number of commons/pack * number of packs/player * number of players
total_uncommons = 4 * 5 * 8 # number of uncommons/pack * number of packs/player * number of players
total_rares = 1 * 5 * 8 # number of rares/pack * number of packs/player * number of players

common_pool_size = 30*4+25
uncommon_pool_size = 30*4+20
rare_pool_size = 20*4+15
cardpool_size = common_pool_size + uncommon_pool_size + rare_pool_size
total_drafted = 13* 8 * 5
print(f"Commons with 3+ copies      {min_copies_probability(common_pool_size, total_commons, min_copies=3, max_copies=20) * 100:.0f}%  est. num of cards:  {min_copies_probability(common_pool_size, total_commons, min_copies=2, max_copies=20) * total_commons}")
print(f"Uncommons with 2+ copies    {min_copies_probability(uncommon_pool_size, total_uncommons, min_copies=2, max_copies=20)*100:.0f}% est. num of cards:   {min_copies_probability(uncommon_pool_size, total_commons, min_copies=2, max_copies=20) * total_uncommons}")
print(f"Rares with 2+ copies        {min_copies_probability(rare_pool_size, total_rares, min_copies=2, max_copies=20) * 100:.0f}% est. num of cards:   {min_copies_probability(rare_pool_size, total_rares, min_copies=2, max_copies=20) * total_rares}")
print(f"{cardpool_size} cards in cardpool")
print(f"Total cards drafted: {total_drafted}")

Commons with 3+ copies      38%  est. num of cards:  207.399006584944
Uncommons with 2+ copies    32% est. num of cards:   106.70548280432305
Rares with 2+ copies        7% est. num of cards:   2.6572008715207813
380 cards in cardpool
Total cards drafted: 520


In [3]:
first_draft = pd.read_csv("corp_spreadsheets/CorpPool_Draft1.csv")
first_draft.shape

(380, 5)

In [1]:
first_draft = pd.read_csv("corp_spreadsheets/CorpPool_Draft1.csv")
# get the number of rarities for each card type
first_draft.groupby(['Faction','Rarity'])['Card'].count().reset_index()

# Need probably 2 more agendas per main faction, 1 more neutrals probably fine, just swaps from common/uncommon

NameError: name 'pd' is not defined

In [4]:
first_draft = pd.read_csv("corp_spreadsheets/CorpPool_Draft1.csv")
# get the number of rarities for each card type
first_draft.query('Type == "Agenda"').groupby(['Faction','Rarity'])['Card'].count().reset_index()

,Faction,Rarity,Card
0,Neutral,C,3
1,Neutral,N,3
2,Neutral,R,3
3,haas-bioroid,C,6
4,haas-bioroid,N,5
5,haas-bioroid,R,3
6,jinteki,C,6
7,jinteki,N,5
8,jinteki,R,3
9,nbn,C,6


In [6]:
def upgrade_rarities(draft_pack: pd.DataFrame, rarity: str, cardpool: pd.DataFrame):
    """
    Upgrade the rarities of the cards in the draft pack
    If the card is an agenda, only upgrade to other agendas
    Otherwise upgrade from any other type to any other type (excluding agendas)
    """
    if rarity not in ['R', 'N']:
        raise ValueError("Rarity must be R or N")
    for row in draft_pack.iterrows():
        if row[1]['Type'] == 'Agenda':
            rare_pool = cardpool[(cardpool['Rarity'] == rarity) & (cardpool['Type'] == row[1]['Type'])].copy()
        else:
            rare_pool = cardpool[(cardpool['Rarity'] == rarity) & (cardpool['Type'] != 'Agenda')].copy()
        rare_card = rare_pool.sample(1)
        row[1]['Card'] = rare_card['Card'].values[0]
        row[1]['Rarity'] = rare_card['Rarity'].values[0]
        row[1]['Faction'] = rare_card['Faction'].values[0]
        row[1]['Type'] = rare_card['Type'].values[0]
        row[1]['Function'] = rare_card['Function'].values[0]
    return draft_pack

def get_commons_baseline(cardpool: pd.DataFrame):
    """
    Get the number of commons in the cardpool

    TODO: Need to put checker for duplicate commons in the same pack
    """
    df = cardpool[cardpool['Rarity'] == 'C'].copy()
    agendas = df[df['Type'] == 'Agenda'].sample(2)
    ice = df[df['Type'] == 'Ice'].sample(3)
    operations = df[df['Type'] == 'Operation'].sample(2)
    assets = df[df['Type'] == 'Asset'].sample(2)
    flex = df.sample(4)
    df = pd.concat([agendas, ice, operations, assets, flex])
    return df

def upgrade_baseline(commons_dataframe: pd.DataFrame, cardpool: pd.DataFrame):
    pack_rares = commons_dataframe.iloc[:1,:]
    pack_uncommons = commons_dataframe.iloc[1:,:]
    pack_rares = upgrade_rarities(pack_rares, 'R', cardpool)
    pack_uncommons = upgrade_rarities(pack_uncommons, 'N', cardpool)
    return commons_dataframe

def faction_checker(pack: pd.DataFrame, faction_list):
    """
    Check to see if the pack has all 5 factions
    """
    for faction in faction_list:
        if pack[pack['Faction'] == faction].shape[0] == 0:
            return faction
    return None

def faction_replace(pack: pd.DataFrame, faction: str, cardpool: pd.DataFrame):
    """
    Replace a card in the pack with a card from the cardpool
    """
    to_be_replaced_card= pack.sample(1)

    card = cardpool[(cardpool['Faction'] == faction) & (cardpool['Rarity'] == to_be_replaced_card['Rarity'].values[0])].sample(1)
    cols = list(pack.columns)
    pack.loc[pack['Card'] == to_be_replaced_card['Card'].values[0], cols] = card[cols].values[0]
    return pack

def faction_swap(pack: pd.DataFrame, cardpool: pd.DataFrame,  faction_list =  ['haas-bioroid', 'jinteki', 'nbn', 'weyland', 'Neutral']):
    while True:
        missing_faction = faction_checker(pack, faction_list)
        if missing_faction is None:
            return pack
        pack = faction_replace(pack, missing_faction, cardpool)
        deduplicator(pack, cardpool)

def deduplicator(pack: pd.DataFrame, cardpool: pd.DataFrame):
    # Find duplicate cards in the pack and replace them with cards of the same rarity
    duplicated_cards = pack[pack.duplicated(subset=['Card'])]
    unique_cards = pack[~pack.duplicated(subset=['Card'])]
    cols = list(pack.columns)
    for row in duplicated_cards.iterrows():
        card = cardpool[(cardpool['Rarity'] == row[1]['Rarity']) & (cardpool['Type'] == row[1]['Type'])].sample(1)
        for col in cols:
            row[1][col] = card[col].values[0]
    new_pack = pd.concat([unique_cards, duplicated_cards])
    if new_pack.duplicated(subset=['Card']).sum() > 0:
        return deduplicator(new_pack, cardpool)
    return new_pack


def make_corp_pack(cardpool: pd.DataFrame):
    """
    13 card packs - with 5 rounds of drafting gets you 65 cards

    Guranteed:
    - 2 Agendas
    - 3 Ice
    - 2 Operations
    - 2 Assets
    - 4 Flex cards

    Rarity Breadown:
    1 Rares
    5 Uncommons
    7 Commons

    From the common pool fill out the guranteed card types
    Randomly upgrade 2 rares and 4 uncommons
    Then check for all factions present - swapping out as needed for the same type
    optional - check for ice duplication
    """
    pack = get_commons_baseline(cardpool).reset_index(drop=True)
    pack_upgrades = pack.sample(6)
    pack_commons = pack[~pack.isin(pack_upgrades)].dropna()
    pack_upgrades = upgrade_baseline(pack_upgrades, cardpool)
    pack = pd.concat([pack_upgrades, pack_commons])
    pack = faction_swap(pack, cardpool)
    return pack

pack = make_corp_pack(first_draft)

In [7]:
make_corp_pack(first_draft)

,Card,Rarity,Type,Faction,Function
2,Drago Ivanov,R,Asset,nbn,TagGen
7,Shipment from MirrorMorph,N,Operation,haas-bioroid,Combo
0,Regulatory Capture,N,Agenda,weyland,BadPub
8,Enigma,N,Ice,Neutral,CodeGate
11,Punitive Counterstrike,N,Operation,weyland,Combo
4,Mr. Hendrik,N,Upgrade,haas-bioroid,CoreDamage
1,Architect Deployment Test,C,Agenda,haas-bioroid,Glacier
3,Aiki,C,Ice,jinteki,CodeGate
5,Beanstalk Royalties,C,Operation,weyland,Economy
6,Launch Campaign,C,Operation,Neutral,AssetSpam


In [563]:
pack = make_corp_pack(first_draft)
pack.groupby(['Faction'])['Card'].count().reset_index().shape[0] != 5

False

In [9]:
pool_counts = pd.concat([make_corp_pack(first_draft).reset_index(drop=True) for i in range(0,5 * 8)]).reset_index(drop=True).groupby(['Card'])['Rarity'].count().reset_index()
pool_counts.rename(columns={'Rarity':'Copies'}, inplace=True)
df = pd.merge(first_draft, pool_counts, how='left', on='Card')
# df.to_csv("corp_spreadsheets\drafted_pool_v1.csv", index=False)


In [10]:
df.groupby("Card").aggregate({'Copies':'sum', 'Rarity':'first'}).reset_index().sort_values(by='Copies', ascending=False)

,Card,Copies,Rarity
246,Oversight AI,6.0,C
277,RSVP,6.0,C
356,Tsurugi,6.0,C
220,Mumba Temple,5.0,C
377,Yagura,5.0,C
...,...,...,...
230,NGO Front,0.0,R
233,Neural EMP,0.0,N
236,Nico Campaign,0.0,C
70,Closed Account,0.0,R


In [11]:
pool_data = df.groupby("Card").aggregate({"Copies":sum, "Type":max, 'Rarity':max}).sort_values(by='Copies', ascending=False).reset_index()
cards_df.columns
df = pd.merge(pool_data[pool_data['Type'] == 'Agenda'], cards_df[['stripped_title','type_code','agenda_points', 'faction_code']].groupby("stripped_title").aggregate({'agenda_points':max, 'faction_code':max}), how='left', left_on='Card', right_on='stripped_title')
df['total_points'] = df['agenda_points'] * df['Copies']
df.groupby('faction_code')['total_points'].sum().reset_index()

,faction_code,total_points
0,haas-bioroid,44.0
1,jinteki,51.0
2,nbn,68.0
3,neutral-corp,32.0
4,weyland-consortium,37.0


In [12]:
df

,Card,Copies,Type,Rarity,agenda_points,faction_code,total_points
0,Reeducation,5.0,Agenda,C,3.0,nbn,15.0
1,Hyperloop Extension,4.0,Agenda,N,1.0,haas-bioroid,4.0
2,Explode-a-palooza,4.0,Agenda,N,2.0,nbn,8.0
3,Regenesis,4.0,Agenda,N,1.0,jinteki,4.0
4,Remote Enforcement,4.0,Agenda,C,2.0,haas-bioroid,8.0
...,...,...,...,...,...,...,...
60,Government Takeover,0.0,Agenda,R,6.0,weyland-consortium,0.0
61,Megaprix Qualifier,0.0,Agenda,C,1.0,haas-bioroid,0.0
62,City Works Project,0.0,Agenda,R,3.0,weyland-consortium,0.0
63,Project Vacheron,0.0,Agenda,R,3.0,haas-bioroid,0.0
